In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
frequency = 60 # Example of value

def radian_frequency(frequency):
    radian_frequency =  2*math.pi*frequency
    return radian_frequency
print(radian_frequency(60))

376.99111843077515


In [3]:
L_1_2  = 0.04/radian_frequency(60)
L_1_3 = 0.03/radian_frequency(60)
L_2_3 = 0.0215/radian_frequency(60)

R_1_2 = 0.02
R_1_3 = 0.01
R_2_3 = 0.0125

zetha = []

for L, R in zip([L_1_2, L_1_3, L_2_3], [R_1_2, R_1_3, R_2_3]):
    zetha.append(R + 1j*radian_frequency(60)* L)


print(zetha)

[(0.02+0.04j), (0.01+0.03j), (0.0125+0.0215j)]


In [4]:
y_vector =  []

for i in range(len(zetha)):
  y_vector.append(1/np.imag(zetha[i]))


print(y_vector)

[25.0, 33.333333333333336, 46.51162790697675]


In [5]:

def calculate_y_bus(R, L, C, n, m):
  y_bus_set = set()

  #def build_admittance_matrices(R, L, C, X_i, X_j, i, j, f, aprox = True):


In [6]:
def build_admittance_matrices(R, L, C, X_i, X_j, i, j, f, aprox = True, mode=1):
    """
    Construye Ybus, B, Bprime (B′) y Bdoubleprime (B″) a partir de una lista de items:
      ('R', i, j, R_ij)           # oh
    Parámetros:
      f: frecuencia en Hz
      n_buses: si None, se infiere del máx índice observado
      one_based: True si los índices vienen 1..N, False si 0..N-1
    Retorna:
      dict con 'Ybus', 'B', 'Bprime', 'Bdoubleprime'
    """
    #n = len(R)
    n = max(max(i),max(j))

    w = 2*np.pi*f
    if mode == 1:
      X = [L[i] * w for i in range(len(L))]
    elif mode == 2:
      X = L
    if aprox:
        Y = [1/(1j*X[i]) for i in range(len(X))]
    else:
        Y = [1/(1j*w*X[i]+R[i]) for i in range(len(R))]
    Y_matrix = np.array([[0j for _ in range(n)] for _ in range(n)])
    print(Y_matrix)
    counter = 0
    for i_index,j_index in zip(i,j):
        print(i_index, j_index)
        Y_matrix[i_index-1][j_index-1] = -Y[counter]
        Y_matrix[j_index-1][i_index-1] = -Y[counter]
        counter += 1
    for i_index in range(len(Y_matrix)):
      for j_index in range(len(Y_matrix)):
          if j_index != i_index:
            Y_matrix[i_index][i_index] -= Y_matrix[i_index][j_index]
    return Y_matrix

In [7]:
R = np.array([0.10,0.05,0.08,0.05, 0.05, 0.10, 0.07, 0.12, 0.02, 0.20, 0.10])
L = np.array([0.20, 0.20, 0.30, 0.25, 0.10, 0.30, 0.20, 0.30, 0.10, 0.40, 0.30])

Y = build_admittance_matrices(R, L, 0,0,0, np.array([1,1,1,2,2,2,2,3,3,4,5]), np.array([2,4,5,3,4,5,6,5,6,5,6]), 60, mode=2)
Y

[[0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]]
1 2
1 4
1 5
2 3
2 4
2 5
2 6
3 5
3 6
4 5
5 6


array([[ 0.-13.33333333j, -0. +5.j        ,  0. +0.j        ,
        -0. +5.j        , -0. +3.33333333j,  0. +0.j        ],
       [-0. +5.j        ,  0.-27.33333333j, -0. +4.j        ,
        -0.+10.j        , -0. +3.33333333j, -0. +5.j        ],
       [ 0. +0.j        , -0. +4.j        ,  0.-17.33333333j,
         0. +0.j        , -0. +3.33333333j, -0.+10.j        ],
       [-0. +5.j        , -0.+10.j        ,  0. +0.j        ,
         0.-17.5j       , -0. +2.5j       ,  0. +0.j        ],
       [-0. +3.33333333j, -0. +3.33333333j, -0. +3.33333333j,
        -0. +2.5j       ,  0.-15.83333333j, -0. +3.33333333j],
       [ 0. +0.j        , -0. +5.j        , -0.+10.j        ,
         0. +0.j        , -0. +3.33333333j,  0.-18.33333333j]])

In [8]:
node_type = {'S': 1,  'L': 2 , 'G': 3}

In [10]:
length_L = 1 #len(node_type['L'])
length_G = 1#len(node_type['G'])

def boolean_b_prime(node_type, i):
  if (i == node_type['L'] or i == node_type['G']) and i != node_type['S']:
    return True
  else:
    return False
def boolean_b_prime_prime(node_type, i):
  if (i == node_type['L']) and i != node_type['S']:
    return True
  else:
    return False


lengh_b_prime = length_L + length_G

B = Y.imag
B_prime = np.array([[0j for _ in range(lengh_b_prime)] for _ in range(lengh_b_prime)])

ii = 0
jj = 0
for i_index in range(len(B)):
  for j_index in range(len(B[0])):
      if boolean_b_prime(node_type, i_index+1) and boolean_b_prime(node_type,j_index+1):
        B_prime[ii][jj] = B[i_index][j_index]
        if ii<lengh_b_prime-1:
          ii += 1
        else:
          ii = 0
          jj += 1


B_prime_prime = np.array([[0j for _ in range(length_L)] for _ in range(length_L)])

ii = 0
jj = 0
for i_index in range(len(B)):
  for j_index in range(len(B[0])):
      if boolean_b_prime_prime(node_type, i_index+1) and boolean_b_prime_prime(node_type,j_index+1):
        B_prime_prime[ii][jj] = B[i_index][j_index]
        if ii<lengh_b_prime-1:
          ii += 1
        else:
          ii = 0
          jj += 1
B_prime_prime

print('b_prima',B_prime)
print('b_prima_prima',B_prime_prime)

b_prima [[-27.33333333+0.j   4.        +0.j]
 [  4.        +0.j -17.33333333+0.j]]
b_prima_prima [[-27.33333333+0.j]]
